CONSTANTS

In [1]:
from __future__ import annotations

import os
from datetime import date, timedelta

import numpy as np
import pandas as pd
from prophet import Prophet

# ========= CONFIG =========
dataset = pd.read_excel("2025 Allianz Datathon Dataset.xlsx", sheet_name="Visitation Data")
cut_df = dataset[(dataset["Year"] < 2025) | ((dataset["Year"] == 2025) & (dataset["Week"] <= 8))]
cut_df.to_csv("visit_data.csv", index=False)


INPUT_CSV  = "visit_data.csv"   
SEASON_WEEKS = 15               
OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

USE_COUNTRY_HOLIDAYS = True
COUNTRY_NAME = "Australia"      
REGRESSOR_MODE = "multiplicative"

HELPER FUNCTIONS

In [33]:

# ---------- Week → date mapping (calendar‑aware) ----------
def week_anchor_date(year: int, week: int) -> pd.Timestamp:
    assert 1 <= week <= SEASON_WEEKS
    start = date(year, 6, 9) 
    return pd.Timestamp(start + timedelta(days=7*(week - 1)))

def season_calendar(year: int, weeks: int = SEASON_WEEKS) -> pd.DatetimeIndex:
    start = date(year, 6, 9)
    return pd.to_datetime([start + timedelta(days=7*i) for i in range(weeks)])

def fit_shape_model_real_dates(df_resort: pd.DataFrame,
                               regressor_cols: list[str],
                               regressor_mode: str = "multiplicative"
                                ) -> Prophet:
    """
    Train Prophet on weekly proportions using REAL calendar dates (ds).
    No trend: we want a stable within‑season shape, modulated by holidays/regressors.
    """
    cols = ["ds", "prop"] + regressor_cols
    train = df_resort[cols].copy()
    train = train.rename(columns={"prop":"y"})

    m = Prophet(
        yearly_seasonality=10,       # allows a date‑anchored within‑year pattern
        weekly_seasonality=False,      # already at weekly granularity
        daily_seasonality=False,
        seasonality_mode="multiplicative",
        n_changepoints=0,              # forbid trend in shape
        changepoint_prior_scale=0.001,
        seasonality_prior_scale=5.0
    )

    #add national holidays
    _add_holidays_safe(m)
    for c in regressor_cols:
        m.add_regressor(c, mode=regressor_mode)
    m.fit(train)
    return m


def forecast_shape_for_year_real_dates(m_shape: Prophet,
                                       year: int,
                                       weeks: int,
                                       future_regs: pd.DataFrame,
                                       regressor_cols: list[str],
                                       ) -> pd.DataFrame:
    """
    Forecast weekly proportions for a season on REAL calendar dates.
    If you have regressor_cols, you must supply those columns in future_regressors
    (aligned on 'ds'). Missing columns are filled with zeros.
    """
    fut = pd.DataFrame({"ds": season_calendar(year, weeks)})
    
    need = ["ds"] + regressor_cols
    miss = [c for c in need if c not in future_regs.columns]
    if miss:
        raise ValueError(f"Missing columns in future_regs: {miss}")

    # dedupe on ds just in case
    fr = future_regs[need].drop_duplicates(subset="ds", keep="last").copy()
    fut = fut.merge(fr, on="ds", how="left")

    # fill any gaps
    for c in regressor_cols:
        fut[c] = pd.to_numeric(fut[c], errors="coerce")
        fut[c] = fut[c].interpolate(limit_direction="both").fillna(0.0)

    fc = (
        m_shape.predict(fut)[["ds","yhat","yhat_lower","yhat_upper"]]
        .rename(columns={"yhat":"prop_hat","yhat_lower":"prop_lo","yhat_upper":"prop_hi"})
    )

    # Valid proportions: non‑negative and sum to 1
    for c in ["prop_hat","prop_lo","prop_hi"]:
        fc[c] = fc[c].clip(lower=0)
        s = fc[c].sum()
        fc[c] = fc[c] / s if s > 0 else np.nan

    fc["Week"] = np.arange(1, weeks+1)
    return fc

# ---------- Shape model on REAL dates ----------
def _add_holidays_safe(m: Prophet) -> None:
    if not USE_COUNTRY_HOLIDAYS:
        return
    # Try a couple of country identifiers; ignore if unsupported
    for cn in [COUNTRY_NAME, "AU", "Australia"]:
        if not cn:
            continue
        try:
            m.add_country_holidays(country_name=cn)
            return
        except Exception:
            continue

def fit_total_model(df_resort_annual: pd.DataFrame) -> Prophet:
    train = df_resort_annual.rename(columns={"Year":"ds","T":"y"}).copy()
    train["ds"] = pd.to_datetime(train["ds"].astype(str) + "-01-01")
    m = Prophet(
        yearly_seasonality=False,     # not meaningful for 1/yr sampling
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode="multiplicative",
        n_changepoints=2,
        changepoint_prior_scale=0.1,
    )
    m.fit(train)
    return m

def forecast_total_for_year(m_total: Prophet, year:int) -> pd.DataFrame:
    fut = pd.DataFrame({"ds": [pd.Timestamp(f"{year}-01-01")]})
    fc = m_total.predict(fut)[["ds","yhat","yhat_lower","yhat_upper"]].rename(
        columns={"yhat":"T_hat","yhat_lower":"T_lo","yhat_upper":"T_hi"}
    )
    fc["Year"] = year
    return fc




SRI Index Data

In [3]:
SRI_CSV = "data_input_sri.csv"  # columns: Resort, Date, SRI, ...
sri_raw = pd.read_csv(SRI_CSV)
sri_raw.columns = [c.strip() for c in sri_raw.columns]
sri_raw["Date"] = pd.to_datetime(sri_raw["Date"])
sri_raw["Year"] = sri_raw["Date"].dt.year
sri_raw = sri_raw.sort_values(["Resort", "Date"])

In [ ]:
def slice_sri_by_anchor(g_year: pd.DataFrame, year: int, column: str) -> pd.DataFrame:
    g_year = g_year.sort_values("Date").reset_index(drop=True).copy()
    june_pos = np.flatnonzero(g_year["Date"].dt.month.values == 6)

    if len(june_pos) >= 2:
        start = int(june_pos[1])  # 2nd June row
    elif len(june_pos) == 1:
        start = int(june_pos[0])  # fallback: only one June row
    else:
        target = pd.Timestamp(f"{year}-06-09")
        start = int((g_year["Date"] - target).abs().values.argmin())  # nearest to Jun-09

    sub = g_year.iloc[start:start + SEASON_WEEKS].copy()
    sub["Week"] = np.arange(1, len(sub) + 1)
    sub["ds"] = [week_anchor_date(year, w) for w in sub["Week"]]
    sub["Year"] = year
    return sub[["Resort", "Year", "Week", "ds", column]]

# Build SRI season table for all resorts/years
sri_seasons = []
for resort, gR in sri_raw.groupby("Resort"):
    for year, gY in gR.groupby("Year"):
        # only build a season if we have at least ~8 rows around mid-year
        if len(gY) == 0:
            continue
        sri_seasons.append(slice_sri_by_anchor(gY, year, "SRI"))

sri_season = pd.concat(sri_seasons, ignore_index=True)


Input Stirling and Charlotte Pass Weather

In [5]:
resort_name_map = {
    "Cabramurra SMHEA AWS": "Selwyn",      # same Snowy Mountains region
    "Falls Creek": "Falls Creek",
    "Mount Baw Baw": "Mt. Baw Baw",
    "Mount Buller": "Mt. Buller",
    "Mount Hotham": "Mt. Hotham",
    "Mount Stirling": "Mt. Stirling",      # synthetic copy of Buller
    "Perisher AWS": "Perisher",
    "Thredbo AWS": "Thredbo"
    # Charlotte Pass was created separately from averaging
}

In [6]:
stirling = (
    sri_season[sri_season["Resort"] == "Mount Buller"]
    .copy()
)
stirling["Resort"] = "Mount Stirling"

# Average Thredbo + Perisher → Charlotte Pass
charlotte = (
    sri_season[sri_season["Resort"].isin(["Thredbo AWS", "Perisher AWS"])]
    .groupby(["Year", "Week", "ds"], as_index=False)
    .agg({"SRI": "mean"})
)

charlotte["Resort"] = "Charlotte Pass"

# Append to the main dataset
sri_season_aug = pd.concat([sri_season, stirling, charlotte], ignore_index=True)

# apply to your dataframe
sri_season_aug["Resort"] = sri_season_aug["Resort"].replace(resort_name_map)


Other Regressor

In [ ]:
r_df = pd.read_csv("./rf_forecasts_mulvar.csv")
r_df.columns = [c.strip() for c in r_df.columns]
r_df["Date"] = pd.to_datetime(r_df["Date"])
r_df["Year"] = r_df["Date"].dt.year
r_df = r_df.sort_values(["Resort", "Date"])
r_df

r_seasons = []
for resort, gR in r_df.groupby("Resort"):
    for year, gY in gR.groupby("Year"):
        # only build a season if we have at least ~8 rows around mid-year
        if len(gY) == 0:
            continue
        r_seasons.append(slice_sri_by_anchor(gY, year, ["TempMax_forecast"]))

sri_season = pd.concat(sri_seasons, ignore_index=True)


,Date,Resort,TempMax_forecast,TempMin_forecast,Rainfall_forecast,Year
0,2025-08-10,Cabramurra SMHEA AWS,3.578702,-0.966110,36.7872,2025
1,2025-08-17,Cabramurra SMHEA AWS,3.675585,-0.865386,58.9960,2025
2,2025-08-24,Cabramurra SMHEA AWS,3.811079,-1.158629,24.8020,2025
3,2025-08-31,Cabramurra SMHEA AWS,4.971911,-0.650614,43.7540,2025
4,2025-09-07,Cabramurra SMHEA AWS,4.989097,-0.780276,35.1664,2025
...,...,...,...,...,...,...
520,2026-12-13,Thredbo AWS,14.851717,6.706330,25.9680,2026
521,2026-12-20,Thredbo AWS,16.424726,6.848148,22.9612,2026
522,2026-12-27,Thredbo AWS,17.225423,8.098186,34.5784,2026
523,2027-01-03,Thredbo AWS,17.278568,7.895190,22.9568,2027


Visitor Data Prep

In [7]:


# ---------- Load & reshape ----------
raw = pd.read_csv(INPUT_CSV)
raw.columns = [c.strip() for c in raw.columns]
assert {"Year","Week"}.issubset(set(raw.columns)), "CSV must have Year and Week."

# resort columns are everything except Year/Week and stray unnamed cols
resort_cols = [c for c in raw.columns if c not in {"Year","Week"} and not c.startswith("Unnamed")]
if not resort_cols:
    raise ValueError("No resort columns detected.")

# build ds from Year/Week using your fixed weekly anchors
raw["ds"] = pd.to_datetime([week_anchor_date(int(y), int(w)) for y, w in zip(raw["Year"], raw["Week"])])
# Long format: one row per resort-week
long = raw.melt(id_vars=["Year","Week","ds"], value_vars=resort_cols,
                var_name="resort", value_name="y").sort_values(["resort","ds"])

# ---------- Proportions (shape targets) ----------
season_totals_obs = (
                    long.groupby(["resort","Year"], as_index=False)["y"]
                        .sum()
                        .rename(columns={"y":"season_total_obs"})
                    )
long = long.merge(season_totals_obs, on=["resort","Year"], how="left")
long["prop"] = long["y"] / long["season_total_obs"].replace(0, np.nan)


# ---------- Annual totals (scale) ----------
annual = long.groupby(["resort","Year"], as_index=False)["y"].sum().rename(columns={"y":"T"})


In [8]:
sri_season_aug["resort"] = sri_season_aug["Resort"].str.strip()
long = long.merge(
    sri_season_aug.rename(columns={"SRI": "sri"})[["resort","Year","Week","ds","sri"]],
    on=["resort","Year","Week","ds"],
    how="left"
)

MODEL PREDICTION

In [16]:
sri_fc = pd.read_csv("SRI_forecast_2026_RF.csv")
sri_fc["Date"] = pd.to_datetime(sri_fc["Date"])
sri_fc["Year"] = sri_fc["Date"].dt.year
sri_fc = sri_fc.sort_values(["Resort","Date"])
sri_2026_all = []
for resort, g in sri_fc.groupby("Resort"):
    sri_2026_all.append(slice_sri_by_anchor(g, year=2026, column="SRI_Forecast"))
sri_2026 = pd.concat(sri_2026_all, ignore_index=True)

stirling = (
    sri_2026[sri_2026["Resort"] == "Mount Buller"]
    .copy()
)
stirling["Resort"] = "Mount Stirling"

# Average Thredbo + Perisher → Charlotte Pass
charlotte = (
    sri_2026[sri_2026["Resort"].isin(["Thredbo AWS", "Perisher AWS"])]
    .groupby(["Year", "Week", "ds"], as_index=False)
    .agg({"SRI_Forecast": "mean"})
)

charlotte["Resort"] = "Charlotte Pass"

# Append to the main dataset
sri_2026_aug = pd.concat([sri_2026, stirling, charlotte], ignore_index=True)

# apply to your dataframe
sri_2026_aug["Resort"] = sri_2026_aug["Resort"].replace(resort_name_map)

sri_2026_aug["resort"] = sri_season_aug["Resort"].str.strip()
sri_2026_aug["sri"] = sri_2026_aug["SRI_Forecast"]

In [34]:

# ---------- Train per resort, recompose ----------
forecast_2026_all = []

for resort, dfr in long.groupby("resort"):
    # --- SHAPE on real dates with optional holidays/regressors
    m_shape = fit_shape_model_real_dates(
        dfr,
        regressor_cols=["sri"],
        regressor_mode=REGRESSOR_MODE
    )
    # sri_future_resort = sri_2026_aug[sri_2026_aug["resort"]==resort][["ds","sri"]].copy()
    fut_regs_2026 = (sri_2026_aug.loc[sri_2026_aug["resort"]==resort, ["ds","SRI_Forecast"]]
                                .rename(columns={"SRI_Forecast":"sri"}))

    shape_2026 = forecast_shape_for_year_real_dates(
        m_shape=m_shape,
        year=2026,
        weeks=SEASON_WEEKS,
        future_regs=fut_regs_2026,
        regressor_cols=["sri"],
    )

    # --- TOTALS model on annual sums
    dfr_annual = annual[annual["resort"] == resort].copy()
    m_total = fit_total_model(dfr_annual)

    # 2026 absolute weekly = shape × total
    fc_total_2026 = forecast_total_for_year(m_total, 2026)
    T_hat_2026, T_lo_2026, T_hi_2026 = map(float, fc_total_2026[["T_hat","T_lo","T_hi"]].iloc[0])

    fc2026 = shape_2026.copy()
    fc2026["resort"] = resort
    
    # Recompose with uncertainty bounds (simple product ignoring covariance)
    fc2026["y_abs_hat"] = fc2026["prop_hat"] * T_hat_2026
    fc2026["y_abs_lo"]  = fc2026["prop_lo"]  * T_lo_2026
    fc2026["y_abs_hi"]  = fc2026["prop_hi"]  * T_hi_2026
    fc2026["Year"] = 2026
    forecast_2026_all.append(fc2026[["resort","ds","Year","Week","prop_hat","prop_lo","prop_hi","y_abs_hat","y_abs_lo","y_abs_hi"]])

# ---------- Concatenate & Save ----------
forecast_2026 = pd.concat(forecast_2026_all, ignore_index=True).sort_values(["resort","ds"])

p2026 = os.path.join(OUT_DIR, "two_stage_forecast_2026_by_resort.csv")

# filled_2025.to_csv(p2025, index=False)
forecast_2026.to_csv(p2026, index=False)

print("Saved:", p2026)

15:45:33 - cmdstanpy - INFO - Chain [1] start processing
15:45:33 - cmdstanpy - INFO - Chain [1] done processing
15:45:33 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
15:45:33 - cmdstanpy - INFO - Chain [1] start processing
15:45:33 - cmdstanpy - INFO - Chain [1] done processing
15:45:33 - cmdstanpy - INFO - Chain [1] start processing
15:45:33 - cmdstanpy - INFO - Chain [1] done processing
15:45:33 - cmdstanpy - INFO - Chain [1] start processing
15:45:33 - cmdstanpy - INFO - Chain [1] done processing
15:45:33 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
15:45:33 - cmdstanpy - INFO - Chain [1] start processing
15:45:33 - cmdstanpy - INFO - Chain [1] done processing
15:45:34 - cmdstanpy - INFO - Chain [1] start processing
15:45:34 - cmdstanpy - INFO - Chain [1] done processing
15:45:34 -

Saved: outputs\two_stage_forecast_2026_by_resort.csv


MODEL EVALUATION

In [ ]:
import numpy as np, pandas as pd

def jensen_shannon(p, q, eps=1e-12):
    p = np.clip(p, eps, 1); q = np.clip(q, eps, 1)
    p /= p.sum(); q /= q.sum(); m = 0.5*(p+q)
    return 0.5*(np.sum(p*np.log(p/m)) + np.sum(q*np.log(q/m))) ** 0.5

def evaluate_shape_loyo(long: pd.DataFrame):
    rows = []
    for resort, dfR in long.groupby("resort"):
        for yr, dfY in dfR.groupby("Year"):

            # actual proportions by week (sum to 1 within year)
            tgt = dfY.sort_values("ds")["y"].to_numpy().astype(float)
            tgt = tgt / tgt.sum() if tgt.sum()>0 else np.full_like(tgt, np.nan)

            # train on other years
            train = dfR[dfR["Year"] < yr].copy()
            if train["Year"].nunique() < 2:  # need at least 2 seasons to generalize
                continue
            m = fit_shape_model_real_dates(
                pd.DataFrame({"ds":train["ds"], "prop":(train["y"]/train.groupby("Year")["y"].transform("sum")).values, "sri": train["sri"]}), regressor_cols=["sri"]
            )
            fut = dfY.sort_values("ds")[["ds","sri"]].copy()
            pred = m.predict(fut)["yhat"].clip(lower=0).to_numpy()
            pred = pred / pred.sum() if pred.sum()>0 else np.full_like(pred, np.nan)

            mae = np.nanmean(np.abs(pred - tgt))
            rmse = np.sqrt(np.nanmean((pred - tgt)**2))
            jsd = jensen_shannon(pred, tgt)
            rows.append({"resort":resort,"year":yr,"shape_mae":mae,"shape_rmse":rmse,"shape_jsd":jsd})
    return pd.DataFrame(rows)


In [48]:
# df = pd.read_csv("./outputs/two_stage_forecast_2026_by_resort.csv")
# df.head()
eval_long = long[long["Year"] != 2025] #2025 not complete data
evaluation = evaluate_shape_loyo(long)
evaluation


15:53:49 - cmdstanpy - INFO - Chain [1] start processing
15:53:54 - cmdstanpy - INFO - Chain [1] done processing
15:53:54 - cmdstanpy - INFO - Chain [1] start processing
15:53:54 - cmdstanpy - INFO - Chain [1] done processing
15:53:55 - cmdstanpy - INFO - Chain [1] start processing
15:53:55 - cmdstanpy - INFO - Chain [1] done processing
15:53:55 - cmdstanpy - INFO - Chain [1] start processing
15:53:55 - cmdstanpy - INFO - Chain [1] done processing
15:53:56 - cmdstanpy - INFO - Chain [1] start processing
15:53:56 - cmdstanpy - INFO - Chain [1] done processing
15:53:57 - cmdstanpy - INFO - Chain [1] start processing
15:53:57 - cmdstanpy - INFO - Chain [1] done processing
15:53:57 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
15:53:57 - cmdstanpy - INFO - Chain [1] start processing
15:53:57 - cmdstanpy - INFO - Chain [1] done processing
15:53:57 - cmdstanpy - INFO - Chain [1] start process

,resort,year,shape_mae,shape_rmse,shape_jsd
0,Charlotte Pass,2016,0.012051,0.015774,0.077175
1,Charlotte Pass,2017,0.006801,0.008011,0.036079
2,Charlotte Pass,2018,0.006511,0.008286,0.034691
3,Charlotte Pass,2019,0.007066,0.008830,0.038425
4,Charlotte Pass,2020,0.053276,0.063455,0.284777
...,...,...,...,...,...
85,Thredbo,2021,0.037443,0.046069,0.166497
86,Thredbo,2022,0.011415,0.014366,0.060759
87,Thredbo,2023,0.008065,0.009524,0.041711
88,Thredbo,2024,0.017650,0.019284,0.085262


In [52]:
evaluation.head(50)

,resort,year,shape_mae,shape_rmse,shape_jsd
0,Charlotte Pass,2016,0.012051,0.015774,0.077175
1,Charlotte Pass,2017,0.006801,0.008011,0.036079
2,Charlotte Pass,2018,0.006511,0.008286,0.034691
3,Charlotte Pass,2019,0.007066,0.008830,0.038425
4,Charlotte Pass,2020,0.053276,0.063455,0.284777
5,Charlotte Pass,2021,0.037598,0.046254,0.168541
6,Charlotte Pass,2022,0.012351,0.015389,0.065143
7,Charlotte Pass,2023,0.008100,0.010343,0.045684
8,Charlotte Pass,2024,0.016849,0.018794,0.083937
9,Charlotte Pass,2025,0.026830,0.035202,0.063665


In [1]:
# Show all rows
pd.set_option("display.max_rows", None)

# Show all columns
pd.set_option("display.max_columns", None)

# Prevent columns from getting cut off horizontally
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

evaluation.head(5)

NameError: name 'pd' is not defined

Visitor Amount Model Evaluation

In [24]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data
df = pd.read_csv("visit_data.csv")
# Aggregate weekly into annual totals per resort
resort_cols = [c for c in df.columns if c not in {"Year","Week"}]
annual = df.groupby("Year")[resort_cols].sum().reset_index()
print()
# Storage for results
rows = []

for resort in resort_cols:
    series = annual[["Year", resort]].dropna()
    series = series.set_index("Year")[resort].sort_index()

    # For each possible forecast year
    for target_year in series.index:
        if target_year >= 2025:  # skip 2025 (future)
            continue

        # Train set = all years before target_year
        train = series.loc[series.index < target_year]

        # Only proceed if at least 5 years of history
        if len(train) < 9:
            continue

        try:
            # Fit ETS (additive trend, damped)
            model = ExponentialSmoothing(
                train,
                trend="add",
                seasonal=None,
                damped_trend=True
            ).fit(optimized=True)

            pred = model.forecast(1).iloc[0]
            actual = series.loc[target_year]

            mae = abs(pred - actual)
            rmse = np.sqrt((pred - actual)**2)
            mape = abs((pred - actual) / actual) * 100 if actual != 0 else np.nan

            rows.append({
                "resort": resort,
                "year": target_year,
                "actual": actual,
                "pred": pred,
                "mae": mae,
                "rmse": rmse,
                "mape": mape
            })
        except Exception as e:
            print(f"Skipping {resort} {target_year} due to error: {e}")

# Collect results
results = pd.DataFrame(rows)

# Average metrics per resort
metrics = results.groupby("resort")[["mae","rmse","mape"]].mean().reset_index()

print("Per-forecast results:\n", results.head())
print("\nAverage metrics per resort:\n", metrics)


c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarn

Per-forecast results:
          resort  year  actual           pred            mae           rmse  \
0   Mt. Baw Baw  2023   96854   67266.912358   29587.087642   29587.087642   
1   Mt. Baw Baw  2024   76964   77569.327144     605.327144     605.327144   
2  Mt. Stirling  2023   10959   13094.625343    2135.625343    2135.625343   
3  Mt. Stirling  2024    9846    8522.872299    1323.127701    1323.127701   
4    Mt. Hotham  2023  398746  281370.888912  117375.111088  117375.111088   

        mape  
0  30.548132  
1   0.786507  
2  19.487411  
3  13.438226  
4  29.436060  

Average metrics per resort:
            resort           mae          rmse       mape
0  Charlotte Pass   2961.551769   2961.551769   8.735773
1     Falls Creek  92498.114652  92498.114652  21.126337
2     Mt. Baw Baw  15096.207393  15096.207393  15.667319
3      Mt. Buller  68667.760898  68667.760898  14.564580
4      Mt. Hotham  91385.547440  91385.547440  27.532882
5    Mt. Stirling   1729.376522   1729.376522 

c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWar

In [25]:
results

,resort,year,actual,pred,mae,rmse,mape
0,Mt. Baw Baw,2023,96854,67266.912358,29587.087642,29587.087642,30.548132
1,Mt. Baw Baw,2024,76964,77569.327144,605.327144,605.327144,0.786507
2,Mt. Stirling,2023,10959,13094.625343,2135.625343,2135.625343,19.487411
3,Mt. Stirling,2024,9846,8522.872299,1323.127701,1323.127701,13.438226
4,Mt. Hotham,2023,398746,281370.888912,117375.111088,117375.111088,29.436060
5,Mt. Hotham,2024,255157,320552.983792,65395.983792,65395.983792,25.629704
6,Falls Creek,2023,439826,339438.018042,100387.981958,100387.981958,22.824476
7,Falls Creek,2024,435492,350883.752654,84608.247346,84608.247346,19.428198
8,Mt. Buller,2023,481230,377952.132841,103277.867159,103277.867159,21.461228
9,Mt. Buller,2024,444157,410099.345363,34057.654637,34057.654637,7.667932
